In [1]:
import pandas as pd
import os
import re
import sys
import numpy as np
from merf import MERF
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 
sns.set_context("poster")
from sklearn.ensemble import RandomForestRegressor
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.merf import MERF
from sklearn.model_selection import train_test_split, KFold
from merf.viz import plot_merf_training_stats

current_dir = os.getcwd() # Get the current working directory
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from em_utils import *

# Create output directory if it doesn't exist
output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_plots/5.combined"
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/5.combined"
os.makedirs(output_dir, exist_ok=True)

# Read in prior DFs

In [ ]:
# Meta / clinical
clin_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/clinical/transformed/aim2/merf_ready/"
test_long = pd.read_csv(os.path.join(clin_dir, 'test_clinical_no_na.csv'))  
train_long = pd.read_csv(os.path.join(clin_dir, 'train_clinical_no_na.csv'))  
full_long = pd.read_csv(os.path.join(clin_dir, 'full_train_clinical.csv'))  
raw_train = pd.read_csv(os.path.join(clin_dir, 'raw__train_clinical.csv'))  
raw_test = pd.read_csv(os.path.join(clin_dir, 'raw_test_clinical.csv'))

print("---------- Read metadata ----------")
m1_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/clinical/transformed/aim2"
#met_test = read_data(m1_dir, "long_a2_test_samples_standard_clinical_feb20.csv")
#met_train = read_data(m1_dir, "long_a2_train_samples_standard_clinical_feb20.csv")
met_test = read_data(m1_dir, "long_a2_test_samples_extra_clinical_feb20.csv")
met_train = read_data(m1_dir, "long_a2_train_samples_extra_clinical_feb20.csv")
#met_full = read_data(m1_dir, "a2_meta_Transformed_standard_clinical.csv")
#met_full_raw = read_data(m1_dir, "a2_meta_not_Transformed_standard_clinical.csv")

# Genetic 
gen_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/genetic/"
gen_test = pd.read_csv(os.path.join(gen_dir, 'genetic_risk_testing.csv'))  
gen_train = pd.read_csv(os.path.join(gen_dir, 'genetic_risk_training.csv')) 

# Taxa
tax_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/taxa/aim2_transformed/genus/"
test_tax_no_na = pd.read_csv(os.path.join(tax_dir, 'aim2_clr_testing_feb20.csv'))
train_tax_no_na = pd.read_csv(os.path.join(tax_dir, 'aim2_clr_training_feb20.csv'))
#full_train_tax = pd.read_csv(os.path.join(tax_dir, 'full_train_tax.csv'))
#full_test_tax = pd.read_csv(os.path.join(tax_dir, 'full_test_tax.csv'))

# Functional
func_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/functional/aim2/"
test_func_no_na = pd.read_csv(os.path.join(func_dir, 'all_clr_testing_feb20.csv'))  # Read test_tax_no_na
train_func_no_na = pd.read_csv(os.path.join(func_dir, 'all_clr_training_feb20.csv'))  # Read train_tax_no_na
#full_train_func = pd.read_csv(os.path.join(func_dir, 'full_train_functional.csv'))  # Read full_train_tax
#full_test_func = pd.read_csv(os.path.join(func_dir, 'full_test_functional.csv'))

# Micom fluxes
micom_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/micom/aim2/"
test_micom_no_na = pd.read_csv(os.path.join(micom_dir, 'flux_all_testing_feb20.csv'))  # Read in test_micom_no_na
train_micom_no_na = pd.read_csv(os.path.join(micom_dir, 'flux_all_training_feb20.csv'))  # Read in train_micom_no_na
#full_train_micom = pd.read_csv(os.path.join(micom_dir, 'full_train_micom.csv'))  # Read in full_train_micom
#full_test_micom = pd.read_csv(os.path.join(micom_dir, 'full_test_micom.csv')) 

In [ ]:
print("meta test = ", met_train.shape)
print("Meta tst columns", met_train.columns.to_list())

print("meta train shape = ", met_test.shape)
print("Meta train df columns", met_test.columns.to_list())

print("genetic df shape = ", gen_train.shape)
print("Genetic df columns", gen_train.columns.to_list())
print("Taxa df shape = ", train_tax_no_na.shape)
print("Taxa df columns", train_tax_no_na.columns.to_list())
print("Functional df shape = ", train_func_no_na.shape)
print("Functional df columns", train_func_no_na.columns.to_list())
print("Micom df shape = ", train_micom_no_na.shape)
print("Micom df columns", train_micom_no_na.columns.to_list())

In [ ]:
print("meta subject_id values:", met_train['subject_id'])
print("meta time values:", met_train['time'])

In [ ]:
print("genetic subject_id values:", gen_train['subject_id'])
print("unnamed column in the gennetic data: ", gen_train['Unnamed: 0'])

In [ ]:
print("taxa all_samples values:", train_tax_no_na['all_samples'])

In [ ]:
print("functional all_samples values:", train_func_no_na['all_samples'])
print("functional time values:", train_func_no_na['time'])
print("functional SampleID values:", train_func_no_na['SampleID'])

In [ ]:
print("Micom all_samples values:", train_micom_no_na['all_samples'])
print("Micom time values:", train_micom_no_na['time'])
print("Micom sample_id values:", train_micom_no_na['sample_id'])

### Merge Micom and Functional data

Micom all_samples column matches Functional Sample.ID
##### For training

In [ ]:
### Convert data types for merging
train_micom_no_na['sample_id'] = train_micom_no_na['sample_id'].astype(str)  # Convert to string
train_func_no_na['SampleID'] = train_func_no_na['SampleID'].astype(str)  # Ensure SampleID is string

print("Micom sample_id values:", train_micom_no_na['sample_id'])
print("Functional SampleID values:", train_func_no_na['SampleID']) 

# Count the number of matching rows between 'sample_id' and 'SampleID'
matching_rows_count = train_micom_no_na['sample_id'].isin(train_func_no_na['SampleID']).sum()
print("Number of matching rows between Micom sample_id and Functional SampleID:", matching_rows_count)

64 samples are in the pathway dataset that are not in the micom dataset 

In [ ]:
### Merge Micom and Functional data
merged_micom_func = pd.merge(train_micom_no_na, train_func_no_na, 
                     left_on='sample_id', right_on='SampleID', 
                     how='inner')
print("Merged micom_func shape: ", merged_micom_func.shape)
print("Merged micom_func columns: ", merged_micom_func.columns.to_list())
print("Merged micom_func id: ", merged_micom_func['sample_id'])
print("Merged micom_func all_samples: ", merged_micom_func['all_samples_y'])

##### For testing

In [ ]:
### Convert data types for merging
test_micom_no_na['sample_id'] = test_micom_no_na['sample_id'].astype(str)  # Convert to string
test_func_no_na['SampleID'] = test_func_no_na['SampleID'].astype(str)  # Ensure SampleID is string

print("Micom sample_id values:", test_micom_no_na['sample_id'])
print("Functional SampleID values:", test_func_no_na['SampleID']) 

# Count the number of matching rows between 'sample_id' and 'SampleID'
matching_rows_count = test_micom_no_na['sample_id'].isin(test_func_no_na['SampleID']).sum()
print("Number of matching rows between Micom sample_id and Functional SampleID:", matching_rows_count)

### Merge Micom and Functional data for testing
test_merged_micom_func = pd.merge(test_micom_no_na, test_func_no_na, 
                     left_on='sample_id', right_on='SampleID', 
                     how='inner')
print("Merged micom_func shape: ", test_merged_micom_func.shape)

13 samples in functional id test set is not in micom test set 

### Merge Micom and Functional data with Taxa data 

Using all_samples and time somehow

For training: 

In [ ]:
### Merge merged_micom_func with taxa_train

micom_func_tax = pd.merge(merged_micom_func, train_tax_no_na, 
                         left_on=['all_samples_y', 'time_y'], 
                         right_on=['all_samples', 'time'], 
                         how='inner')

print("Final merged shape: ", micom_func_tax.shape)
print("Micom, taxa and functional df sample_id: ", micom_func_tax['sample_id'])
print("Micom, taxa and functional df all_sampls: ", micom_func_tax['all_samples'])

299 all sample variables (duplicates included)

For testig: 

In [13]:
test_micom_func_tax = pd.merge(test_merged_micom_func, test_tax_no_na, 
                         left_on=['all_samples_y', 'time_y'], 
                         right_on=['all_samples', 'time'], 
                         how='inner')

In [ ]:
print("Final merged columns: ", test_micom_func_tax.columns.to_list())

### Merge genetic data 
for training

In [ ]:
# Create subject_id column by modifying sample_id
micom_func_tax['subject_id'] = micom_func_tax['sample_id'].str.split('.').str[0]  # Remove '.' and following characters
print("Subject_id column to match that of genetic data: ", micom_func_tax['subject_id'])

In [ ]:
print("gen_train columns: ", gen_train.columns.to_list())
gen_train = gen_train[['subject_id', 'bmi_prs']]
print("gen train subject_id: ", gen_train['subject_id'])

In [ ]:
merged_data = pd.merge(micom_func_tax, 
                       gen_train, 
                       on='subject_id', 
                       how='left')

print("Merged data shape: ", merged_data.shape)
print("Merged data columns: ", merged_data.columns.to_list())
# Count unique subject_id values
unique_subject_ids_gen = gen_train['subject_id'].nunique()
print("Number of unique subject_id values in merged: ", unique_subject_ids_gen)

unique_subject_ids_count = merged_data['subject_id'].nunique()
print("Number of unique subject_id values in merged: ", unique_subject_ids_count)

In [ ]:
# Get subject_ids from both DataFrames
gen_train_subject_ids = set(gen_train['subject_id'])
meta_train_subject_ids = set(train_long['subject_id'])
micom_func_tax_subject_ids = set(micom_func_tax['subject_id'])

# Find subject_ids in meta not in genetic data
subject_ids_in_meta_not_in_gen = meta_train_subject_ids - gen_train_subject_ids
print("Subject IDs in gen_train but not in micom_func_tax:", subject_ids_in_meta_not_in_gen)

subject_ids_in_meta_not_in_merged = meta_train_subject_ids - micom_func_tax_subject_ids
print("Subject IDs in meta but not in micom_func_tax:", subject_ids_in_meta_not_in_merged)

# Find subject_ids in gen_train but not in micom_func_tax
subject_ids_in_gen_not_in_micom = gen_train_subject_ids - micom_func_tax_subject_ids
print("Subject IDs in gen_train but not in micom_func_tax:", subject_ids_in_gen_not_in_micom)

# Find subject_ids in micom_func_tax but not in gen_train
subject_ids_in_micom_not_in_gen = micom_func_tax_subject_ids - gen_train_subject_ids
print("Subject IDs in micom_func_tax but not in gen_train:", subject_ids_in_micom_not_in_gen)

#### For testing 

In [ ]:
print("gen_test columns: ", gen_test.columns.to_list())
gen_test = gen_test[['subject_id', 'bmi_prs']]
print("gen train subject_id: ", gen_test['subject_id'])

In [ ]:
# Create subject_id column by modifying sample_id
test_micom_func_tax['subject_id'] = test_micom_func_tax['sample_id'].str.split('.').str[0]  # Remove '.' and following characters
print("Subject_id column to match that of genetic data: ", test_micom_func_tax['subject_id'])

In [ ]:
test_merged_data = pd.merge(test_micom_func_tax, 
                       gen_test, 
                       on='subject_id', 
                       how='left')

print("Merged data shape: ", test_merged_data.shape)
print("Merged data columns: ", test_merged_data.columns.to_list())
# Count unique subject_id values
unique_subject_ids_gen = gen_test['subject_id'].nunique()
print("Number of unique subject_id values in merged: ", unique_subject_ids_gen)

unique_subject_ids_count = test_merged_data['subject_id'].nunique()
print("Number of unique subject_id values in merged: ", unique_subject_ids_count)

### merge meta data to other omics 

for training

In [ ]:
print("Meta train columns: ", met_train.columns.to_list())
print("Meta train subject_id column: ", met_train['subject_id'])
print("test_micom_func_tax subject_id column: ", merged_data['subject_id'])

merged_data["time"] = merged_data["time"].astype(int)
met_train["time"] = met_train["time"].astype(int)
print("test_micom_func_tax time column: ", merged_data['time'])
print("met_train time column: ", met_train['time'])

# Count matching subject_id values
matching_subjects = merged_data['subject_id'].isin(met_train['subject_id']).sum()
print("Number of matching subject_id values:", matching_subjects)


In [ ]:
merged_all = pd.merge(met_train, merged_data, 
                         left_on=['subject_id', 'time'], 
                         right_on=['subject_id', 'time'], 
                         how='inner',
                         suffixes=('_train_long', '_merged_data'))

print("Merged all shape: ", merged_all.shape)
print("Merged all columns: ", merged_all.columns.to_list())
# Count unique subject_id values
unique_subject_ids_all = merged_all['subject_id'].nunique()
print("Number of unique subject_id values in merged_all: ", unique_subject_ids_all)

#### For testing 

In [ ]:
test_merged_all = pd.merge(met_test, 
                       test_merged_data, 
                        left_on=['subject_id', 'time'], 
                        right_on=['subject_id', 'time'],
                        how='inner',
                        suffixes=('_test_long', '_merged_data'))  # Specify suffixes for duplicate columns

print("Merged all shape: ", test_merged_all.shape)
print("Merged all columns: ", test_merged_all.columns.to_list())
# Count unique subject_id values
unique_subject_ids_all = test_merged_all['subject_id'].nunique()
print("Number of unique subject_id values in merged_all: ", unique_subject_ids_all)

In [25]:
# Remove specified columns from merged_all
merged_all = merged_all.drop(columns=['Unnamed: 0_train_long', 'Unnamed: 0_x', 'all_samples_x', 'time_x', 
                                       'Unnamed: 0_y', 'SampleID_x', 'SampleID_y'])

# Remove specified columns from test_merged_all
test_merged_all = test_merged_all.drop(columns=['Unnamed: 0_test_long', 'Unnamed: 0_x', 'all_samples_x', 'time_x', 
                                               'Unnamed: 0_y', 'SampleID_x', 'SampleID_y'])

# Save combined training and testing files 

In [26]:
#merged_all.to_csv(os.path.join(df_dir, 'feb20_training_merged_all_omics_raw_meta.csv'), index=False)  # Save merged_all to CSV
#test_merged_all.to_csv(os.path.join(df_dir, 'feb20_test_merged_all_omics_raw_meta.csv'), index=False)  # Save test_merged_all to CSV

merged_all.to_csv(os.path.join(df_dir, 'feb20_training_merged_all_omics_extra_meta.csv'), index=False)  # Save merged_all to CSV
test_merged_all.to_csv(os.path.join(df_dir, 'feb20_test_merged_all_omics_extra_meta.csv'), index=False)